In [17]:
# 라이브러리 호출
import webbrowser
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.alert import Alert 

# 데이터 직군 공고 크롤링
- 함수생성

In [18]:
# 직군 별 채용공고 모든 url 추출하기 ex) 개발 > 데이터엔지니어 직군 전체 공고

def web_url_list(url):
    browser = webdriver.Chrome()
    browser.get(url) # 직업군 분류 url주소

    time.sleep(2)

    before_h = browser.execute_script("return window.scrollY") # execute_script : 자바명령어 실행

    # 원티드 무한스크롤 끝까지 내리기

    while True: #무한반복
        # 맨 아래로 스크롤을 내린다.
        browser.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END)
        # 스크롤 사이 페이지 로딩시간고려 약간의 텀 주기
        time.sleep(3)

        # 스크롤 후 높이
        after_h = browser.execute_script("return window.scrollY")
        
        # 끝까지 내려간 경우 stop
        if after_h == before_h:
            break
        before_h = after_h


    # 스크롤 끝까지 내린 후 모든 공고의 url 추출
    urls = browser.find_elements(By.CSS_SELECTOR, ".Card_className__u5rsb > a")

    url_list = []
    for url in urls:
        url = url.get_attribute("href")
        url_list.append(url)

    print(url_list)
    print(len(url_list))

    return url_list

# 총 738개 url 추출 완료

In [19]:
bigdata_eng = "https://www.wanted.co.kr/wdlist/518/1025?country=kr&job_sort=job.latest_order&years=-1&locations=all"
data_sci = "https://www.wanted.co.kr/wdlist/518/1024?country=kr&job_sort=job.latest_order&years=-1&locations=all"
data_analy = "https://www.wanted.co.kr/wdlist/507/656?country=kr&job_sort=job.latest_order&years=-1&locations=all"

In [20]:
url_bigdata_eng = web_url_list(bigdata_eng)

['https://www.wanted.co.kr/wd/183274', 'https://www.wanted.co.kr/wd/183049', 'https://www.wanted.co.kr/wd/182955', 'https://www.wanted.co.kr/wd/182913', 'https://www.wanted.co.kr/wd/182617', 'https://www.wanted.co.kr/wd/182615', 'https://www.wanted.co.kr/wd/182506', 'https://www.wanted.co.kr/wd/182362', 'https://www.wanted.co.kr/wd/182130', 'https://www.wanted.co.kr/wd/182124', 'https://www.wanted.co.kr/wd/181917', 'https://www.wanted.co.kr/wd/181514', 'https://www.wanted.co.kr/wd/181471', 'https://www.wanted.co.kr/wd/181337', 'https://www.wanted.co.kr/wd/181098', 'https://www.wanted.co.kr/wd/180837', 'https://www.wanted.co.kr/wd/180586', 'https://www.wanted.co.kr/wd/180491', 'https://www.wanted.co.kr/wd/180362', 'https://www.wanted.co.kr/wd/180030', 'https://www.wanted.co.kr/wd/179903', 'https://www.wanted.co.kr/wd/179020', 'https://www.wanted.co.kr/wd/178983', 'https://www.wanted.co.kr/wd/178648', 'https://www.wanted.co.kr/wd/178342', 'https://www.wanted.co.kr/wd/178341', 'https://ww

In [21]:
url_data_sci = web_url_list(data_sci)

['https://www.wanted.co.kr/wd/183352', 'https://www.wanted.co.kr/wd/183023', 'https://www.wanted.co.kr/wd/182996', 'https://www.wanted.co.kr/wd/182955', 'https://www.wanted.co.kr/wd/182913', 'https://www.wanted.co.kr/wd/182902', 'https://www.wanted.co.kr/wd/182867', 'https://www.wanted.co.kr/wd/182633', 'https://www.wanted.co.kr/wd/182617', 'https://www.wanted.co.kr/wd/182615', 'https://www.wanted.co.kr/wd/182506', 'https://www.wanted.co.kr/wd/182407', 'https://www.wanted.co.kr/wd/182404', 'https://www.wanted.co.kr/wd/182362', 'https://www.wanted.co.kr/wd/182293', 'https://www.wanted.co.kr/wd/182235', 'https://www.wanted.co.kr/wd/182124', 'https://www.wanted.co.kr/wd/182075', 'https://www.wanted.co.kr/wd/181975', 'https://www.wanted.co.kr/wd/181974', 'https://www.wanted.co.kr/wd/181973', 'https://www.wanted.co.kr/wd/181815', 'https://www.wanted.co.kr/wd/181814', 'https://www.wanted.co.kr/wd/181509', 'https://www.wanted.co.kr/wd/181505', 'https://www.wanted.co.kr/wd/181253', 'https://ww

In [22]:
url_data_analy = web_url_list(data_analy)

['https://www.wanted.co.kr/wd/183137', 'https://www.wanted.co.kr/wd/183068', 'https://www.wanted.co.kr/wd/182821', 'https://www.wanted.co.kr/wd/182735', 'https://www.wanted.co.kr/wd/182725', 'https://www.wanted.co.kr/wd/182682', 'https://www.wanted.co.kr/wd/182387', 'https://www.wanted.co.kr/wd/182310', 'https://www.wanted.co.kr/wd/182237', 'https://www.wanted.co.kr/wd/182220', 'https://www.wanted.co.kr/wd/182096', 'https://www.wanted.co.kr/wd/181986', 'https://www.wanted.co.kr/wd/181909', 'https://www.wanted.co.kr/wd/181684', 'https://www.wanted.co.kr/wd/181619', 'https://www.wanted.co.kr/wd/181615', 'https://www.wanted.co.kr/wd/181464', 'https://www.wanted.co.kr/wd/181406', 'https://www.wanted.co.kr/wd/180945', 'https://www.wanted.co.kr/wd/180776', 'https://www.wanted.co.kr/wd/180654', 'https://www.wanted.co.kr/wd/180596', 'https://www.wanted.co.kr/wd/180345', 'https://www.wanted.co.kr/wd/180330', 'https://www.wanted.co.kr/wd/180221', 'https://www.wanted.co.kr/wd/179973', 'https://ww

# 채용공고 -> 지역, 우대사항, tool 크롤링
- 함수생성

In [31]:
def prefer_list(url_list): # 채용공고의 url list입력 시 우대사항 정보 및 크롤링 실패한 url 주소 반환
    success = 0 # 성공횟수
    fail = 0 # 실패횟수

    name = []
    loc = []
    prefer = []
    tool = []
    urls = []
    fail_url = [] # 크롤링에 실패 한 url 저장

    for url in url_list:
        try :
            browser = webdriver.Chrome()
            browser.get(url) # 크롤링 한 채용공고의 url    
            
            urls.append(url)
            names = browser.find_elements(By.CSS_SELECTOR, "section.JobHeader_className__HttDA a")
            location = browser.find_element(By.CSS_SELECTOR, "div.JobContent_className___ca57 span.JobHeader_pcLocationContainer__xRwIv").text
            details = browser.find_elements(By.CSS_SELECTOR, "div.JobContent_descriptionWrapper__SM4UD p")
            # JobContent_descriptionWrapper__SM4UD 클래스의 하위 p > 하위 span태그 추출

            name.append(names[0].text) # 회사 이름
            loc.append(location) # 태그의 1번째가 지역. -> 1번째면 loc 리스트에 저장
            prefer.append(details[3].text) # span태그의 3번째가 우대사항. -> 3번째면 prefer 리스트에 저장

            # tool의 경우 없는 공고도 있으므로 try, except문 ..
            try :
                tool.append(details[5].text) # span태그의 5번째가 tool. -> 5번째면 tool 리스트에 저장
            except :
                tool.append(None)
            fail_url.append(None)

            browser.quit()
            success += 1
            print("success_count : ", success)

        except :
            fail += 1
            name.append(None)
            loc.append(None) 
            prefer.append(None) 
            tool.append(None) 
            urls.append(None)
            fail_url.append(url)
            browser.quit()
            print("fail_count : ", fail)
    return name, loc, prefer, tool, urls, fail_url

# 함수 호출 및 csv저장

In [32]:
name, loc1, prefer, tool, urls, fail_url = prefer_list(url_bigdata_eng)
pd.DataFrame(name).to_csv("name_bigdata_eng.csv", index = False)
pd.DataFrame(loc1).to_csv("loc_bigdata_eng.csv", index = False)
pd.DataFrame(prefer).to_csv("prefer_bigdata_eng.csv", index = False)
pd.DataFrame(tool).to_csv("tool_bigdata_eng.csv", index = False)
pd.DataFrame(urls).to_csv("urls_bigdata_eng.csv", index = False)
pd.DataFrame(fail_url).to_csv("fail_url_bigdata_eng.csv", index = False)

name, loc2, prefer, tool, urls, fail_url = prefer_list(url_data_sci)
pd.DataFrame(name).to_csv("name_data_sci.csv", index = False)
pd.DataFrame(loc2).to_csv("loc_data_sci.csv", index = False)
pd.DataFrame(prefer).to_csv("prefer_data_sci.csv", index = False)
pd.DataFrame(tool).to_csv("tool_data_sci.csv", index = False)
pd.DataFrame(urls).to_csv("urls_data_sci.csv", index = False)
pd.DataFrame(fail_url).to_csv("fail_url_data_sci.csv", index = False)

name, loc3, prefer, tool, urls, fail_url = prefer_list(url_data_analy)
pd.DataFrame(name).to_csv("name_data_analy.csv", index = False)
pd.DataFrame(loc3).to_csv("loc_data_analy.csv", index = False)
pd.DataFrame(prefer).to_csv("prefer_data_analy.csv", index = False)
pd.DataFrame(tool).to_csv("tool_data_analy.csv", index = False)
pd.DataFrame(urls).to_csv("urls_data_analy.csv", index = False)
pd.DataFrame(fail_url).to_csv("fail_url_data_analy.csv", index = False)

success_count :  1
success_count :  2
success_count :  3
success_count :  4
success_count :  5
success_count :  6
success_count :  7
success_count :  8
success_count :  9
success_count :  10
success_count :  11
success_count :  12
success_count :  13
success_count :  14
success_count :  15
success_count :  16
success_count :  17
success_count :  18
success_count :  19
success_count :  20
success_count :  21
success_count :  22
success_count :  23
success_count :  24
success_count :  25
success_count :  26
success_count :  27
success_count :  28
success_count :  29
success_count :  30
success_count :  31
success_count :  32
success_count :  33
success_count :  34
success_count :  35
success_count :  36
success_count :  37
success_count :  38
success_count :  39
success_count :  40
success_count :  41
success_count :  42
success_count :  43
success_count :  44
success_count :  45
success_count :  46
success_count :  47
success_count :  48
success_count :  49
success_count :  50
success_c

In [33]:
print(loc1)

['서울.한국', '대전.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '경기.한국', '서울.한국', '서울.한국', '경기.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '경기.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '경기.한국', '경기.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '경기.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '인천.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '경기.한국', '서울.한국', '서울.한국', '서울.한국', '경기.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '경기.한국', '경기.한국', '경기.한국', '경기.한국', '서울.한국', '경기.한국', '경기.한국', '서울.한국', '서울.한국', '대전.한국', '대전.한국', '경남.한국', '대전.한국', '서울.한국', '서울.한국', '서울.한국', '경기.한국', '서울.한국', '경기.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '서울.한국', '경기.한국', '경기.한국', '서울.한국', '경기.한국', 

In [36]:
pd.DataFrame(loc1).to_csv("loc_bigdata_eng.csv", index = False)
pd.DataFrame(loc2).to_csv("loc_data_sci.csv", index = False)
pd.DataFrame(loc3).to_csv("loc_data_analy.csv", index = False)